读取数据

In [55]:
import pandas as pd

In [56]:
train_json = pd.read_json('../train_dataset/train.json')

In [57]:
train_json['annotations'][0]

{'filename': 'train_images\\00001.jpg',
 'period': 'Morning',
 'weather': 'Cloudy'}

In [58]:
# 改成图片的路径格式
train_json['filename'] = train_json['annotations'].apply(lambda x : x['filename'].replace('\\', '/'))
train_json['period'] = train_json['annotations'].apply(lambda x : x['period'])
train_json['weather'] = train_json['annotations'].apply(lambda x : x['weather'])


In [59]:
train_json.head()

,annotations,filename,period,weather
0,"{'filename': 'train_images\00001.jpg', 'period...",train_images/00001.jpg,Morning,Cloudy
1,"{'filename': 'train_images\00002.jpg', 'period...",train_images/00002.jpg,Afternoon,Cloudy
2,"{'filename': 'train_images\00003.jpg', 'period...",train_images/00003.jpg,Morning,Cloudy
3,"{'filename': 'train_images\00004.jpg', 'period...",train_images/00004.jpg,Morning,Sunny
4,"{'filename': 'train_images\00005.jpg', 'period...",train_images/00005.jpg,Afternoon,Cloudy


标签编码

In [60]:
# 标签编码
train_json['period'], period_dict = pd.factorize(train_json['period'])
train_json['weather'], weather_dict = pd.factorize(train_json['weather'])

In [61]:
period_dict,weather_dict

(Index(['Morning', 'Afternoon', 'Dawn', 'Dusk'], dtype='object'),
 Index(['Cloudy', 'Sunny', 'Rainy'], dtype='object'))

统计标签

In [62]:
train_json['period'].value_counts()

0    1613
1     829
3     124
2      34
Name: period, dtype: int64

In [63]:
train_json['weather'].value_counts()

0    1119
1     886
2     595
Name: weather, dtype: int64

对数据进行处理，首先得到数据，包括数据路径和数据的标签，对标签进行编码，对标签进行统计，部分标签对应数据过少，可对数据进行增强。

In [64]:
train_json

,annotations,filename,period,weather
0,"{'filename': 'train_images\00001.jpg', 'period...",train_images/00001.jpg,0,0
1,"{'filename': 'train_images\00002.jpg', 'period...",train_images/00002.jpg,1,0
2,"{'filename': 'train_images\00003.jpg', 'period...",train_images/00003.jpg,0,0
3,"{'filename': 'train_images\00004.jpg', 'period...",train_images/00004.jpg,0,1
4,"{'filename': 'train_images\00005.jpg', 'period...",train_images/00005.jpg,1,0
...,...,...,...,...
2595,"{'filename': 'train_images\02996.jpg', 'period...",train_images/02996.jpg,1,1
2596,"{'filename': 'train_images\02997.jpg', 'period...",train_images/02997.jpg,0,2
2597,"{'filename': 'train_images\02998.jpg', 'period...",train_images/02998.jpg,0,2
2598,"{'filename': 'train_images\02999.jpg', 'period...",train_images/02999.jpg,0,2


对数据进行预处理

In [65]:
from torchvision import transforms as T
from PIL import Image
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
import numpy as np

In [66]:
class WeatherDataset(Dataset):
    def __init__(self, df):
        super(WeatherDataset, self).__init__()
        self.df = df

        self.transform = T.Compose([
            T.Resize(size=(100, 100)),
            T.RandomCrop(size=(64, 64)),  # 随即裁剪
            T.RandomRotation(10),   # 在(-10, 10)之间旋转一定角度
            T.RandomHorizontalFlip(),   # 按照0.5的概率对图片进行水平翻转
            T.RandomVerticalFlip(),     # 按照0.5的概率对图片进行垂直翻转
            T.ToTensor(),
            T.Normalize((0.5,), (0.5,))
        ])

    def __getitem__(self, index):
        file_name = '../train_dataset/' + self.df['filename'].iloc[index]
        img = Image.open(file_name)
        img = self.transform(img)
        return img, torch.tensor(self.df['period'].iloc[index]), torch.tensor(self.df['weather'].iloc[index])
    def __len__(self):
        return len(self.df)

In [67]:
train_dataset = WeatherDataset(train_json.iloc[:-500])
val_dataset = WeatherDataset(train_json.iloc[-500:])

In [68]:
train_dataset[0][0]
# import cv2
# # image = cv2.imread()
# cv2.imshow('image', train_dataset[0][0].numpy())
# cv2.waitKey(0)

tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]]])

In [69]:
i = 0
for x, y1, y1 in train_dataset:
    i+=1
    print(x.shape)
    if i == 10:
        break

torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])


In [70]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

模型搭建

In [48]:
class WeatherModel(nn.Module):
    def __init__(self):
        super(WeatherModel, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 96, 11, 4),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(6400, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )
        self.fc1 = nn.Linear(10, 4)
        self.fc2 = nn.Linear(10, 3)

    def forward(self, x):
        out = self.conv(x)
        fc = self.fc(out)
        logist1 = self.fc1(fc)
        logist2 = self.fc2(fc)
        return logist1, logist2

resnet

In [71]:
import torch.nn.functional as F
from d2l import torch as d2l
import d2l

In [82]:
# 单个残差块
class Residual(nn.Module):
    def __init__(self, in_channels, out_channels,use_1x1conv=False, stride=1):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels,kernel_size=3,padding=1, stride=stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
    def forward(self,X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return F.relu(Y + X)

In [83]:
net = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

In [84]:
def resnet_block(in_channels, out_channels, num_residuals, first_block=False):
    if first_block:
        assert in_channels == out_channels # 第一个模块的通道数和输入通道数一致
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=2))
        else:
            blk.append(Residual(out_channels, out_channels))
    return nn.Sequential(*blk)

In [85]:
net.add_module('resnet_block1', resnet_block(64, 64, 2, first_block=True))
net.add_module('resnet_block2', resnet_block(64, 128, 2))
net.add_module('resnet_block3', resnet_block(128, 256, 2))
net.add_module('resnet_block4', resnet_block(256, 512, 2))

In [86]:
net.add_module('global_avg_pool', nn.AdaptiveAvgPool2d((1, 1)))
net.add_module('fc', nn.Sequential(nn.Flatten(),nn.Linear(512, 10)))

In [87]:
class Resnet(nn.Module):
    def __init__(self, net):
        super(Resnet, self).__init__()
        self.resnet = net
        self.fc1 = nn.Linear(10, 4)
        self.fc2 = nn.Linear(10, 3)

    def forward(self, x):
        fc = self.resnet(x)
        logist1 = self.fc1(fc)
        logist2 = self.fc2(fc)
        return logist1, logist2

In [88]:
model = Resnet(net)
model(torch.tensor(np.random.rand(10, 3, 64, 64).astype(np.float32)))

(tensor([[-0.4257, -0.3799, -0.2771, -0.6143],
         [-0.5950, -0.3372, -0.1151, -0.7659],
         [-0.4305, -0.3472, -0.0273, -0.7403],
         [-0.5373, -0.2308, -0.0327, -0.6800],
         [-0.6122, -0.3049, -0.1851, -0.5699],
         [-0.5881, -0.3708, -0.1312, -0.7301],
         [-0.3884, -0.4392, -0.2811, -0.4558],
         [-0.4902, -0.3314, -0.0932, -0.5333],
         [-0.4184, -0.4402, -0.1186, -0.6483],
         [-0.4304, -0.5231, -0.3442, -0.6360]], grad_fn=<AddmmBackward>),
 tensor([[ 0.2695, -0.3989, -0.4391],
         [ 0.0912, -0.0531, -0.4529],
         [ 0.0395,  0.0250, -0.2708],
         [ 0.0369, -0.2127, -0.4743],
         [ 0.3085, -0.2855, -0.5756],
         [ 0.1680, -0.1040, -0.5745],
         [ 0.2825, -0.2198, -0.3281],
         [ 0.1078, -0.0704, -0.3495],
         [ 0.2706, -0.1155, -0.3780],
         [ 0.0592, -0.2060, -0.3371]], grad_fn=<AddmmBackward>))

In [16]:
model = WeatherModel()
model(torch.tensor(np.random.rand(10, 3, 224, 224).astype(np.float32)))

(tensor([[-0.2455,  0.1682, -0.1396,  0.0274],
         [-0.2413,  0.1612, -0.1377,  0.0333],
         [-0.2410,  0.1672, -0.1421,  0.0310],
         [-0.2385,  0.1656, -0.1373,  0.0261],
         [-0.2299,  0.1703, -0.1443,  0.0255],
         [-0.2295,  0.1755, -0.1448,  0.0419],
         [-0.2431,  0.1761, -0.1399,  0.0323],
         [-0.2344,  0.1711, -0.1449,  0.0349],
         [-0.2385,  0.1760, -0.1510,  0.0359],
         [-0.2410,  0.1631, -0.1322,  0.0277]], grad_fn=<AddmmBackward>),
 tensor([[ 0.2052, -0.2900, -0.2383],
         [ 0.2053, -0.2857, -0.2382],
         [ 0.2077, -0.2870, -0.2365],
         [ 0.2046, -0.2813, -0.2401],
         [ 0.2085, -0.2813, -0.2352],
         [ 0.2179, -0.2713, -0.2376],
         [ 0.2059, -0.2716, -0.2391],
         [ 0.2104, -0.2767, -0.2390],
         [ 0.2065, -0.2758, -0.2404],
         [ 0.2014, -0.2870, -0.2359]], grad_fn=<AddmmBackward>))

模型训练

In [89]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [90]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
for epoch in range(10):
    train_loss, val_loss = [], []
    train_acc1, train_acc2 = [], []
    val_acc1, val_acc2 = [], []

    model.train()
    for i, (x, y1, y2) in enumerate(train_loader):
        x = x.to(device)
        y1 = y1.to(device)
        y2 = y2.to(device)
        optimizer.zero_grad()
        pred1, pred2 = model(x)
        loss = criterion(pred1, y1) + criterion(pred2, y2)
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        train_acc1.append((pred1.argmax(1) == y1.flatten()).cpu().numpy().mean())
        train_acc2.append((pred2.argmax(1) == y2.flatten()).cpu().numpy().mean())

    model.eval()
    for i,(x, y1, y2) in enumerate(val_loader):
        x = x.to(device)
        y1 = y1.to(device)
        y2 = y2.to(device)
        pred1, pred2 = model(x)
        loss = criterion(pred1, y1) + criterion(pred2, y2)
        val_loss.append(loss.item())
        val_acc1.append((pred1.argmax(1) == y1.flatten()).cpu().numpy().mean())
        val_acc2.append((pred2.argmax(1) == y2.flatten()).cpu().numpy().mean())

    if epoch % 1 == 0:
        print(f'\nEpoch:{epoch}')
        print(f'Loss:{np.mean(train_loss):3.5f}/{np.mean(val_loss):3.5f}')
        print(f'period acc:{np.mean(train_acc1):3.5f}/{np.mean(val_acc1):3.5f}')
        print(f'weather acc:{np.mean(train_acc2):3.5f}/{np.mean(val_acc2):3.5f}')


Epoch:0
Loss:1.89421/1.81712
period acc:0.61032/0.57166
weather acc:0.58428/0.57497

Epoch:1
Loss:1.56799/1.60033
period acc:0.61648/0.60472
weather acc:0.69934/0.62861

Epoch:2
Loss:1.48690/1.67217
period acc:0.62263/0.55709
weather acc:0.72775/0.64934

Epoch:3
Loss:1.44197/1.77010
period acc:0.64820/0.61929
weather acc:0.75095/0.56280

Epoch:4
Loss:1.37810/1.42768
period acc:0.65956/0.66316
weather acc:0.76089/0.71139

Epoch:5
Loss:1.29845/1.42596
period acc:0.67330/0.69922
weather acc:0.77462/0.67142

Epoch:6
Loss:1.28466/1.60822
period acc:0.67140/0.67924
weather acc:0.78977/0.62410

Epoch:7
Loss:1.25570/1.50190
period acc:0.68229/0.67593
weather acc:0.79782/0.71319

Epoch:8
Loss:1.26722/1.51606
period acc:0.67708/0.65325
weather acc:0.78693/0.67203

Epoch:9
Loss:1.16584/1.54040
period acc:0.70739/0.69651
weather acc:0.80208/0.63131


In [91]:
path = 'alexnet.pth'
# torch.save(model.state_dict(), path)
model.load_state_dict(torch.load(path))

FileNotFoundError: [Errno 2] No such file or directory: 'alexnet.pth'

In [92]:
model(torch.tensor(np.random.rand(10, 3, 224, 224).astype(np.float32)))

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

预测与提交

In [93]:
import glob
test_df = pd.DataFrame({'filename': glob.glob('../test_dataset/test_images/*.jpg')})
test_df['period'] = 0
test_df['weather'] = 0
test_df = test_df.sort_values(by = 'filename')

In [94]:
test_dataset = WeatherDataset(test_df)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [95]:
model.eval()
period_pred = []
weather_pred = []
for i, (x, y1, y2) in enumerate(test_dataloader):
    x = x.to(device)
    y1 = y1.to(device)
    y2 = y2.to(device)
    pred1, pred2 = model(x)
    period_pred += period_dict[pred1.argmax(1).cpu().numpy()].tolist()
    weather_pred += weather_dict[pred2.argmax(1).cpu().numpy()].tolist()

test_df['period'] = period_pred
test_df['weather'] = weather_pred

In [96]:
test_df

,filename,period,weather
0,../test_dataset/test_images\00008.jpg,Morning,Cloudy
1,../test_dataset/test_images\00018.jpg,Morning,Cloudy
2,../test_dataset/test_images\00020.jpg,Morning,Cloudy
3,../test_dataset/test_images\00022.jpg,Morning,Sunny
4,../test_dataset/test_images\00035.jpg,Afternoon,Cloudy
...,...,...,...
395,../test_dataset/test_images\02970.jpg,Morning,Sunny
396,../test_dataset/test_images\02973.jpg,Morning,Cloudy
397,../test_dataset/test_images\02976.jpg,Morning,Cloudy
398,../test_dataset/test_images\02988.jpg,Morning,Cloudy


In [97]:
import json

In [98]:
submit_json = {'annotations':[]}
for row in test_df.iterrows():
    submit_json['annotations'].append({
        'filename':'test_images\\' + row[1].filename.split('\\')[-1],
        'period':row[1].period,
        'weather':row[1].weather
    })
with open('submit.json', 'w') as up:
    json.dump(submit_json, up)